In [ ]:
import pandas as pd
import numpy as np
import torch
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import squareform

# 设置全局GPU设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用设备：{device}（GPU加速已启用）")


# ----------------------
# 1. 数据预处理（CPU→GPU，仅一次转换）
# ----------------------
def preprocess_data(raw_data):
    """
    预处理原始数据，转为GPU张量格式
    返回：
        close_tensor: GPU张量，形状 [股票数量, 日期数量]，值为收盘价
        stock_codes: 股票代码列表（与张量索引对应）
        dates: 日期列表
    """
    # 基础清洗（CPU上完成，仅一次）
    df = raw_data[['date', 'stock_code', 'close', 'paused']].copy()
    df['date'] = pd.to_datetime(df['date'])
    df = df[df['paused'] != 1].drop(columns='paused')
    df = df[df['date'].dt.year < 2025].dropna(subset=['close'])
    
    # 转为透视表（日期×股票，便于转换为张量）
    pivot_df = df.pivot(index='date', columns='stock_code', values='close')
    stock_codes = pivot_df.columns.tolist()  # 股票代码列表
    dates = pivot_df.index.tolist()          # 日期列表
    
    # 填充缺失值（用前一天数据，GPU上填充效率低，先在CPU完成）
    pivot_df = pivot_df.fillna(method='ffill').fillna(method='bfill')
    
    # 转换为GPU张量（形状 [日期数, 股票数] → 转置为 [股票数, 日期数] 便于按股票处理）
    close_tensor = torch.tensor(pivot_df.values, dtype=torch.float32, device=device).T
    
    print(f"数据预处理完成：{len(stock_codes)}只股票，{len(dates)}个交易日，已加载至{device}")
    return close_tensor, stock_codes, dates


# ----------------------
# 2. 周期趋势特征计算（全GPU加速）
# ----------------------
def gpu_calculate_cycle_features(close_tensor, cycle_days=5):
    """
    GPU批量计算所有股票的周期特征（避免循环）
    close_tensor: 形状 [股票数, 日期数]
    返回：
        cycle_return: 形状 [股票数, 有效日期数]，周期涨跌幅
        up_days_ratio: 形状 [股票数, 有效日期数]，周期内上涨天数占比
    """
    n_stocks, n_dates = close_tensor.shape
    valid_dates = n_dates - cycle_days + 1  # 有效周期数
    
    # ----------------------
    # 特征1：周期涨跌幅（GPU向量化计算）
    # 公式：(周期末收盘价 / 周期初收盘价) - 1
    # ----------------------
    # 提取每个周期的起始价（[:, 0:end-周期+1]）和结束价（[:, 周期-1:end]）
    start_prices = close_tensor[:, :n_dates - cycle_days + 1]
    end_prices = close_tensor[:, cycle_days - 1:]
    cycle_return = (end_prices / start_prices) - 1  # 广播机制批量计算
    
    # ----------------------
    # 特征2：周期内上涨天数占比（GPU向量化计算）
    # ----------------------
    # 先计算单日涨跌幅（close[t]/close[t-1] - 1）
    daily_returns = (close_tensor[:, 1:] / close_tensor[:, :-1]) - 1  # 形状 [股票数, 日期数-1]
    
    # 用滑动窗口统计每个周期内上涨天数（>0的天数）
    # 方法：将涨跌幅>0转为1，否则0，再用滑动窗口求和
    up_days = (daily_returns > 0).float()  # 形状 [股票数, 日期数-1]
    
    # 滑动窗口求和（使用PyTorch的unfold+sum，比循环快100倍+）
    # unfold(1, cycle_days, 1) → 形状 [股票数, 有效周期数, 周期天数]
    up_days_window = up_days.unfold(dimension=1, size=cycle_days, step=1)
    up_days_ratio = up_days_window.sum(dim=2) / cycle_days  # 求和后除以周期天数
    
    # 确保两个特征的有效日期数一致（取较小值）
    min_valid = min(cycle_return.shape[1], up_days_ratio.shape[1])
    cycle_return = cycle_return[:, :min_valid]
    up_days_ratio = up_days_ratio[:, :min_valid]
    
    print(f"{cycle_days}日周期特征计算完成（GPU）：形状 {cycle_return.shape}")
    return cycle_return, up_days_ratio


# ----------------------
# 3. 周期协同矩阵计算（全GPU加速）
# ----------------------
def gpu_cycle_correlation(cycle_return, up_days_ratio, weight_return=0.6):
    """
    基于周期特征计算协同矩阵（GPU内完成，不回传CPU）
    协同得分 = 权重×周期涨跌幅相关性 + (1-权重)×趋势一致性相关性
    """
    n_stocks = cycle_return.shape[0]
    
    # 标准化函数：(x - 均值) / 标准差（GPU批量处理）
    def normalize(x):
        mean = x.mean(dim=1, keepdim=True)  # 按股票维度算均值
        std = x.std(dim=1, keepdim=True) + 1e-8  # 避免除0
        return (x - mean) / std
    
    # 标准化两个特征
    norm_return = normalize(cycle_return)          # 形状 [股票数, 有效日期数]
    norm_up = normalize(up_days_ratio)             # 形状 [股票数, 有效日期数]
    
    # 计算相关性矩阵（利用矩阵乘法，GPU并行效率极高）
    # 相关性 = (x·y) / (n-1) （x和y已标准化）
    n_periods = norm_return.shape[1]
    corr_return = torch.matmul(norm_return, norm_return.T) / (n_periods - 1)  # [股票数, 股票数]
    corr_up = torch.matmul(norm_up, norm_up.T) / (n_periods - 1)              # [股票数, 股票数]
    
    # 综合协同矩阵（GPU内完成加权）
    cycle_corr_matrix = weight_return * corr_return + (1 - weight_return) * corr_up
    cycle_corr_matrix = torch.clamp(cycle_corr_matrix, -1, 1)  # 限制范围
    
    print(f"周期协同矩阵计算完成（GPU）：{cycle_corr_matrix.shape}")
    return cycle_corr_matrix


# ----------------------
# 4. 聚类与结果处理（复用逻辑，仅最后一步转回CPU）
# ----------------------
def hierarchical_cycle_groups(cycle_corr_matrix, stock_codes, min_corr=0.4, min_group_size=3):
    """从GPU协同矩阵中筛选高协同组（仅矩阵数据转回CPU）"""
    # 仅将协同矩阵转回CPU（其他计算仍在GPU）
    corr_np = cycle_corr_matrix.cpu().numpy()
    
    # 后续聚类逻辑与之前一致（scipy不支持GPU，需在CPU完成）
    distance_matrix = 1 - corr_np
    np.fill_diagonal(distance_matrix, 0)
    distance_matrix = np.maximum(distance_matrix, 0)
    dist_array = squareform(distance_matrix)
    
    linkage_matrix = linkage(dist_array, method='complete')
    max_allowed_distance = 1 - min_corr
    cluster_labels = fcluster(linkage_matrix, t=max_allowed_distance, criterion='distance')
    
    label_to_stocks = {}
    for stock_idx, label in enumerate(cluster_labels):
        label_to_stocks.setdefault(label, []).append(stock_codes[stock_idx])
    
    high_cycle_groups = [
        stocks for stocks in label_to_stocks.values() 
        if len(stocks) >= min_group_size
    ]
    high_cycle_groups.sort(key=lambda x: len(x), reverse=True)
    return high_cycle_groups, corr_np  # 返回CPU版矩阵用于后续查询


# ----------------------
# 5. 主流程（全GPU加速）
# ----------------------
if __name__ == "__main__":
    # 1. 加载原始数据并预处理（CPU→GPU，仅一次）
    raw_daily = pd.read_parquet(r'D:\workspace\xiaoyao\data\stock_daily_price.parquet')
    close_tensor, stock_codes, dates = preprocess_data(raw_daily)
    
    # 2. 计算多周期协同组（5日和10日）
    for cycle_days in [5, 10]:
        print(f"\n===== 开始{cycle_days}日周期计算（全GPU加速） =====")
        
        # 2.1 GPU计算周期特征（无循环，向量化操作）
        cycle_return, up_days_ratio = gpu_calculate_cycle_features(close_tensor, cycle_days=cycle_days)
        
        # 2.2 GPU计算协同矩阵（矩阵乘法并行加速）
        cycle_corr_matrix = gpu_cycle_correlation(cycle_return, up_days_ratio)
        
        # 2.3 聚类得到高协同组（仅此时将矩阵转回CPU）
        high_cycle_groups, corr_np = hierarchical_cycle_groups(
            cycle_corr_matrix, 
            stock_codes, 
            min_corr=0.4, 
            min_group_size=3
        )
        corr_df = pd.DataFrame(corr_np, index=stock_codes, columns=stock_codes)
        
        # 2.4 保存结果
        save_path = f"./gpu_high_cycle_groups_{cycle_days}d.csv"
        group_result = []
        for group_id, group in enumerate(high_cycle_groups, 1):
            group_corr = corr_df.loc[group, group]
            upper_tri = group_corr.values[np.triu_indices_from(group_corr, k=1)]
            group_result.append({
                "周期天数": cycle_days,
                "组号": group_id,
                "股票数": len(group),
                "平均协同得分": round(upper_tri.mean(), 4),
                "股票代码": ",".join(group)
            })
        pd.DataFrame(group_result).to_csv(save_path, index=False, encoding="utf-8-sig")
        print(f"{cycle_days}日结果已保存至：{save_path}")
        
        # 2.5 查找目标股票的协同组
        target_stock = "600570.XSHG"
        if target_stock in stock_codes:
            target_idx = stock_codes.index(target_stock)
            # 从高协同组中找目标股票所在组
            target_group = next((g for g in high_cycle_groups if target_stock in g), None)
            if target_group:
                print(f"\n{cycle_days}日周期中，{target_stock}的高协同组（{len(target_group)}只股票）：")
                print("前5只高协同股票：", target_group[:5])
            else:
                print(f"\n{cycle_days}日周期中，未找到{target_stock}的高协同组")
        else:
            print(f"\n{target_stock}不在股票池中")

单日数据清洗完成：时间范围 2005-01-04 00:00:00 至 2024-12-31 00:00:00，包含 5383 只股票

===== 开始计算5日周期趋势协同 =====


KeyboardInterrupt: 